In [1]:
import os
os.chdir("D:\Python\Industry level\Deep learning\Chicken disease classification")
%pwd

<>:2: SyntaxWarning: invalid escape sequence '\P'
<>:2: SyntaxWarning: invalid escape sequence '\P'
C:\Users\Ankit\AppData\Local\Temp\ipykernel_18764\2764130293.py:2: SyntaxWarning: invalid escape sequence '\P'
  os.chdir("D:\Python\Industry level\Deep learning\Chicken disease classification")


'D:\\Python\\Industry level\\Deep learning\\Chicken disease classification'

In [2]:
import tensorflow as tf

d:\Python\Industry level\Deep learning\Chicken disease classification\chicken\Lib\site-packages\google\protobuf\runtime_version.py:98: UserWarning: Protobuf gencode version 5.28.3 is exactly one major version older than the runtime version 6.31.1 at tensorflow/core/framework/attr_value.proto. Please update the gencode to avoid compatibility violations in the next runtime release.
  warnings.warn(
d:\Python\Industry level\Deep learning\Chicken disease classification\chicken\Lib\site-packages\google\protobuf\runtime_version.py:98: UserWarning: Protobuf gencode version 5.28.3 is exactly one major version older than the runtime version 6.31.1 at tensorflow/core/framework/tensor.proto. Please update the gencode to avoid compatibility violations in the next runtime release.
  warnings.warn(
d:\Python\Industry level\Deep learning\Chicken disease classification\chicken\Lib\site-packages\google\protobuf\runtime_version.py:98: UserWarning: Protobuf gencode version 5.28.3 is exactly one major ver

In [5]:
model = tf.keras.models.load_model("artifacts/training/trained_model.keras")

In [6]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    params_image_size: list
    params_batch_size: int

In [7]:
from CNN_Classifier.constants import *
from CNN_Classifier.utils.common import read_yaml,create_directories,save_json

In [8]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH        
    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    def get_evaluation_config(self):
        eval_config = EvaluationConfig(
            path_of_model= "artifacts/training/trained_model.keras",
            training_data= "artifacts/data_ingestion/Chicken-fecal-images",
            all_params= self.params,
            params_image_size= self.params.IMAGE_SIZE,
            params_batch_size= self.params.BATCH_SIZE
        )

        return eval_config
    

In [9]:
from urllib.parse import urlparse

In [10]:
class Evaluation:
    def __init__(self,config: EvaluationConfig):
        self.config = config

    def _valid_generator(self):
        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split = 0.30
        )

        dataflow_kwargs = dict(
            target_size = self.config.params_image_size[:-1],
            batch_size = self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory = self.config.training_data,
            subset = "validation",
            shuffle= False,
            **dataflow_kwargs
        )

    @staticmethod
    def load_model(path:Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
        
    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = model.evaluate(self.valid_generator)

    def save_score(self):
        scores = {"loss":self.score[0],"accuracy":self.score[1]}
        save_json(path=Path("scores.json"),data=scores)

In [11]:
try:
    config = ConfigurationManager()
    val_config = config.get_evaluation_config()
    evaluation= Evaluation(val_config)
    evaluation.evaluation()
    evaluation.save_score()
except Exception as e:
    raise e

[2025-07-30 19:02:36,747: INFO: yaml file: config\config.yaml loaded successfully]
[2025-07-30 19:02:36,786: INFO: yaml file: params.yaml loaded successfully]
[2025-07-30 19:02:36,797: INFO: Created directory at: artifacts]
Found 116 images belonging to 2 classes.


d:\Python\Industry level\Deep learning\Chicken disease classification\chicken\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


8/8 ━━━━━━━━━━━━━━━━━━━━ 42s 5s/step - accuracy: 0.5000 - loss: 9.1756
[2025-07-30 19:03:23,179: INFO: json file saved at: scores.json]
